In [ ]:
!pip3 install beautifulsoup4
!pip3 install lxml
!pip3 install html5lib
!pip3 install grequests
!pip3 install gevent

In [1]:
#csv function definitions:

import csv


def saveSetToFile(my_set, csv_file_name):
    for url in my_set:
        appendRowToCsvFile(url, csv_file_name)
        
        
def appendRowToCsvFile(csvRow, csv_file_name):
    with open(csv_file_name, 'a') as f:
        writer = csv.writer(f)
        writer.writerow([csvRow])

        
def loadCsvToSet(my_set, csv_file_name):
    mycsv = csv.reader(open(csv_file_name))
    for row in mycsv:
        url = row[0]
        my_set.add(url)

        
#saveSetToFile(crawled_links, csv_file_name)



In [2]:
#url helper functions
from bs4 import BeautifulSoup
import requests
import urllib.robotparser as robotparser
from urllib.parse import urlparse
from urllib.parse import urljoin


def does_url_exist(url):
    try: 
        r = requests.head(url)
        if r.status_code < 400:
            return True
        else:
            return False
    except requests.exceptions.RequestException as e:
        #print(e)
        return False

    
def getUrlBase(url):
    url_parsed = urlparse(url)
    return (url_parsed.scheme + '://' + url_parsed.netloc)
    
        
def hasRobotText(url):
    return does_url_exist(urljoin(getUrlBase(url), 'robots.txt'))
                          

def canFetchUrl(url):
    rp = robotparser.RobotFileParser()
    can_fetch = False
    robotUrl = urljoin(getUrlBase(url), 'robots.txt')          
    if hasRobotText(robotUrl):
        try:
            rp.set_url(robotUrl) #robotparser get address
            rp.read() #robotparser read  
            can_fetch = rp.can_fetch("*",url) # true if we can fetch the url according to robot.txt
        except BaseException as e:
            print("An exception ocurred : %s" % str(e)) 
            #print(url)
            #print("")
            #print(". . .") 
    else: can_fetch = True #no robot means everything allowed
    return can_fetch


def getLinksFromPage(url):
    page_links = [] #links in this page
        
    resp = requests.get(url)
    html = BeautifulSoup(resp.text, 'html.parser') #get html
    for a in html.find_all('a'):
        link = a.get('href')
        if isinstance(link, str) and link.startswith('http'):
            link = link.lower()
            appendRowToCsvFile(link, csv_file_name)
            page_links.append(link) #extract href url
    return page_links
    

        

In [3]:
import time

#import grequests


#init:
#initial_url_address = 'https://machinelearningmastery.com/compare-performance-machine-learning-algorithms-weka/'
#initial_url_address = 'http://guidesarchive.ign.com/guides/10798/secrets.html'
initial_url_address = 'https://es.wikipedia.org/wiki/Resident_Evil_2' 
current_batch = [initial_url_address] #initialize crawl batch
crawled_links = {initial_url_address} #hold the overall url links repo
max_depth = 5 #how deep is the breadth-first approach
max_crawl_pages = 5

csv_file_name = "crawlResultsCsv_" + str(round(time.time())) + ".csv" #csv unique name

print("starting the crawler...")
print("saving results on %s" %csv_file_name)
        

#TODO: 
# 1. Cache robots.txt, otherwise it is constatntly requesting!
# 2. Make asyncronous
  
    
crawled_pages = 0
    
# main loop
for i in range(max_depth):
    print("")
    print(". . .")
    print("step: %s" % (i+1))
    print("batch size: %s" % len(current_batch))
    print("crawled unique links: %s" % len(crawled_links))
    print("")
    print("")

    next_batch = []
    
    for url in current_batch:
        print("")
        print("fetching page %s" % crawled_pages + ", it might take a while : %s" % url)
        print("")
        
        page_links = getLinksFromPage(url)
        crawled_pages =+1
        if (crawled_pages > max_crawl_pages):
            break
        else: 
            for link in page_links:
                if link not in crawled_links:
                    if canFetchUrl(link):
                        #Here we only save the allowed links for the next batch
                        crawled_links.add(link)
                        appendRowToCsvFile(link, csv_file_name)
                        next_batch.append(link) #since it was a new link, we will crawl it in the next batch                    
                    #else: print("not allowed to fetch : %s" % link)  
        current_batch = []
        current_batch = next_batch
    

    
print("")
print(". . .")
print("FINISH!")
print("unique links size: %s" % len(crawled_links))

starting the crawler...
saving results on crawlResultsCsv_1549551725.csv

. . .
step: 1
batch size: 1
crawled unique links: 1



fetching page 0, it might take a while : https://es.wikipedia.org/wiki/Resident_Evil_2

An exception ocurred : 'utf-8' codec can't decode byte 0xe9 in position 64972: invalid continuation byte

. . .
step: 2
batch size: 162
crawled unique links: 163



fetching page 1, it might take a while : https://www.wikidata.org/wiki/q733992?uselang=es#p348


fetching page 1, it might take a while : https://www.wikidata.org/wiki/q733992?uselang=es#p479


fetching page 1, it might take a while : http://www.capcom.co.jp/bio_series/biohazard2.html


fetching page 1, it might take a while : http://www.imdb.com/title/tt0161941/


fetching page 1, it might take a while : https://www.wikidata.org/wiki/q733992


fetching page 1, it might take a while : http://www.hobbyconsolas.com/reportajes/biohazard-resident-evil-juegos-con-varios-nombres-68074


fetching page 1, it might take

KeyboardInterrupt: 

In [5]:
# Optional, for loading the results CSV:
test_set={initial_url_address}
loadCsvToSet(test_set, csv_file_name)

print(len(test_set))


533


In [5]:
## test robots.txt
from urllib.parse import urlparse

test1 = 'https://www.buzzfeed.com/buzzfeed/api/' #not allowed by robot.txt
test2 = 'https://github.com/lluissuros?tab=overview&from=2019-02-01&to=2019-02-07' # not allowed?
test3 = 'https://slate.com/culture/sports' #allowed
test4 = 'http://www.capcom.co.jp/newproducts/consumer/value/' #no robots.txt, therefore allowed


print(hasRobotText(test1))
print(hasRobotText(test2))
print(hasRobotText(test3))
print(hasRobotText(test4))

print("..")

print(canFetchUrl(test1))
print(canFetchUrl(test2))
print(canFetchUrl(test3))
print(canFetchUrl(test4))




True
True
True
False
..
False
False
True
True
